In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import style
import random 
import nltk
#!pip install tweet-preprocessor
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.optimizers import Adam

import numpy as np
from numpy import array
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv1D, Flatten,MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

In [ ]:
ds= pd.read_csv('/content/drive/MyDrive/179k_pos_neg.csv', encoding = "ISO-8859-1",header=None)

ds.columns = ["text", "polarity","label"]

ds.head()
 

,text,polarity,label
0,if i smelled the scent of hand sanitizers toda...,-0.25,0
1,hey and wouldnt it have made more sense to hav...,0.50,1
2,trump never once claimed covid19 was a hoax we...,0.00,1
3,the one gift covid19 has give me is an appreci...,0.00,1
4,25 july media bulletin on novel coronavirusupd...,0.00,1


In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [ ]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [ ]:
ds['text']=ds['text'].str.replace('[^\w\s]',"")
ds['text']

0         if i smelled the scent of hand sanitizers toda...
1         hey and wouldnt it have made more sense to hav...
2         trump never once claimed covid19 was a hoax we...
3         the one gift covid19 has give me is an appreci...
4         25 july media bulletin on novel coronavirusupd...
                                ...                        
178945    thanks for nominating me for the wearamask cha...
178946                2020 the year of insanity lol covid19
178947    a powerful painting by juan lucena its a tribu...
178948    more than 1200 students test positive for covi...
178949                     i stop when i see a stopentsabc1
Name: text, Length: 178950, dtype: object

In [ ]:
ds['text']=ds['text'].str.replace('[^a-zA-Z#]'," ")
ds['text']


0         if i smelled the scent of hand sanitizers toda...
1         hey and wouldnt it have made more sense to hav...
2         trump never once claimed covid   was a hoax we...
3         the one gift covid   has give me is an appreci...
4            july media bulletin on novel coronavirusupd...
                                ...                        
178945    thanks for nominating me for the wearamask cha...
178946                     the year of insanity lol covid  
178947    a powerful painting by juan lucena its a tribu...
178948    more than      students test positive for covi...
178949                     i stop when i see a stopentsabc 
Name: text, Length: 178950, dtype: object

In [ ]:
ds['text']=ds['text'].apply(lambda x : " ".join ([x for x in x.split() if len(x)>3]))
ds['text']

0         smelled scent hand sanitizers today someone pa...
1         wouldnt have made more sense have players thei...
2         trump never once claimed covid hoax claim that...
3         gift covid give appreciation simple things tha...
4         july media bulletin novel coronavirusupdates c...
                                ...                        
178945      thanks nominating wearamask challengei nominate
178946                                  year insanity covid
178947    powerful painting juan lucena tribute grandpar...
178948    more than students test positive covid major u...
178949                                stop when stopentsabc
Name: text, Length: 178950, dtype: object

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop=stopwords.words('english')
ds['text']=ds['text'].apply(lambda x : " ".join (x for x in x.split() if x not in stop))

ds['text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0         smelled scent hand sanitizers today someone pa...
1                       wouldnt made sense players respects
2               trump never claimed covid hoax claim effort
3         gift covid give appreciation simple things alw...
4         july media bulletin novel coronavirusupdates c...
                                ...                        
178945      thanks nominating wearamask challengei nominate
178946                                  year insanity covid
178947    powerful painting juan lucena tribute grandpar...
178948    students test positive covid major university ...
178949                                     stop stopentsabc
Name: text, Length: 178950, dtype: object

In [ ]:
from textblob import TextBlob
from textblob import Word

ds['text']=ds['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

ds['text']

0         smelled scent hand sanitizers today someone pa...
1                         wouldnt made sense player respect
2               trump never claimed covid hoax claim effort
3         gift covid give appreciation simple thing alwa...
4         july medium bulletin novel coronavirusupdates ...
                                ...                        
178945      thanks nominating wearamask challengei nominate
178946                                  year insanity covid
178947    powerful painting juan lucena tribute grandpar...
178948    student test positive covid major university news
178949                                     stop stopentsabc
Name: text, Length: 178950, dtype: object

In [ ]:
text = ds['text'].tolist()
text[:3]

['smelled scent hand sanitizers today someone past would think intoxicated',
 'wouldnt made sense player respect',
 'trump never claimed covid hoax claim effort']

In [ ]:
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word

ds['text']=ds['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
text = ds['text'].tolist()
text[:3]

['smelled scent hand sanitizers today someone past would think intoxicated',
 'wouldnt made sense player respect',
 'trump never claimed covid hoax claim effort']

In [ ]:
y = ds['label']

In [ ]:
token = Tokenizer()
token.fit_on_texts(text)

In [ ]:
vocab_size  = len(token.word_index) + 1
vocab_size

112670

In [ ]:
encoded_text = token.texts_to_sequences(text)

In [ ]:
print(encoded_text[:3])

[[21344, 10249, 167, 2940, 12, 251, 315, 57, 71, 40042], [1795, 149, 771, 609, 1239], [19, 158, 1941, 1, 757, 429, 382]]


In [ ]:
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [ ]:
X.shape

(178950, 120)

In [ ]:
glove_vectors = dict()

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-04-21 20:04:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-21 20:04:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-21 20:04:40--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
file = open('/content/glove.6B.200d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1: ])
    
    glove_vectors[word] = vectors
file.close()

In [ ]:
keys = glove_vectors.keys()
len(keys)

400000

In [ ]:
word_vector_matrix = np.zeros((vocab_size, 200))

for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
        print(word)

Streaming output truncated to the last 5000 lines.
diagnosisfrequent
urbansketchers
fastameal
bienvenidosabordoviernesdevinosmaravillamartinezdynamitebtsuelfinalicecreamsmackdown
homecc
safeover
unofficialhalloween
creativecovid
quarantinecreatives
warnedschoolsreopening
plannedhttps
cardiometabolic
conditionssuch
pressurehad
cupwinning
citygo
cupzoom
industrymusic
howtodoitright
stopchildren
soundeverybody
lashdripp
lashextensions
humansdamnwhat
loyali
hospitalizationsuperspreader
areasawaninews
pitchperfect
foreverychild
thewho
anonuncements
auspolhttps
todayheroic
punchnazis
coviddenial
firstweekback
lakhpeopletested
skydying
foreverhappy
deathstoll
weekendhow
raceethnic
sasd
schoolwhata
herpeope
mediacovid
hkgenome
newswho
equitabl
southdakoda
churchlinked
northe
barscovid
youpeo
massivewinds
punekars
tuckin
collabuae
primaryrocks
youpeope
recordmiaminative
californiafire
nonaturaldisasters
melbin
oneissue
ohiohighschool
ncsea
listenfollow
ncsealeads
casabonita
allocatio
morningfou

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3, stratify = y)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
vec_size = 200
voc_size=5000
sent_length=20

embedding_vector_features=40

model = Sequential()

model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights = [word_vector_matrix], trainable = False))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
model.add(Dropout(0.5))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy',f1_m,precision_m,recall_m])



In [ ]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
3915/3915 [==============================] - 498s 127ms/step - loss: 0.3165 - accuracy: 0.8735 - f1_m: 0.9259 - precision_m: 0.8942 - recall_m: 0.9627 - val_loss: 0.3221 - val_accuracy: 0.8760 - val_f1_m: 0.9284 - val_precision_m: 0.8847 - val_recall_m: 0.9792
Epoch 2/10
3915/3915 [==============================] - 514s 131ms/step - loss: 0.3056 - accuracy: 0.8787 - f1_m: 0.9286 - precision_m: 0.9000 - recall_m: 0.9617 - val_loss: 0.3140 - val_accuracy: 0.8763 - val_f1_m: 0.9287 - val_precision_m: 0.8829 - val_recall_m: 0.9821
Epoch 3/10
3915/3915 [==============================] - 506s 129ms/step - loss: 0.2969 - accuracy: 0.8830 - f1_m: 0.9309 - precision_m: 0.9047 - recall_m: 0.9613 - val_loss: 0.3134 - val_accuracy: 0.8791 - val_f1_m: 0.9302 - val_precision_m: 0.8861 - val_recall_m: 0.9815
Epoch 4/10
3915/3915 [==============================] - 526s 134ms/step - loss: 0.2892 - accuracy: 0.8868 - f1_m: 0.9330 - precision_m: 0.9090 - recall_m: 0.9608 - val_loss: 0.3073 - v

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.8886281251907349:
f1_score = 0.935003936290741
Precision = 0.8985915184020996
Recall = 0.9769447445869446


In [ ]:
!pip install -q pyyaml h5py
model.save('Covid_Hybrid.h5')